[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/maxar-open-data/blob/master/examples/morocco_earthquake.ipynb)
[![image](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/opengeos/maxar-open-data&urlpath=lab/tree/maxar-open-data/examples/morocco_earthquake&branch=master)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/maxar-open-data/blob/master/examples/morocco_earthquake.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeos/maxar-open-data/master?urlpath=lab%2Ftree%2Fexamples)

**Visualizing Maxar Open Data for the 2023 Morocco Earthquake with Leafmap**

The [Maxar Open Data Program](https://www.maxar.com/open-data) provides pre- and post-event high-resolution satellite imagery in support of emergency planning, risk assessment, monitoring of staging areas and emergency response, damage assessment, and recovery. Check out the links below for more information.
- [Maxar Open Data Program](https://www.maxar.com/open-data)
- [Maxar Open Data on AWS](https://registry.opendata.aws/maxar-open-data/)
- [Maxar Open Data on STAC Index](https://stacindex.org/catalogs/maxar-open-data-catalog-ard-format#/)
- [Maxar Open Data on STAC Browser](https://radiantearth.github.io/stac-browser/#/external/maxar-opendata.s3.amazonaws.com/events/catalog.json?.language=en)

The Maxar Open Data STAC catalog URL is: https://maxar-opendata.s3.amazonaws.com/events/catalog.json


In [ ]:
# !pip install -U leafmap geopandas cogeo-mosaic

In [ ]:
import leafmap.foliumap as leafmap

Retrieve all collections from the Maxar Open Data STAC catalog. Each collection represents a single event.

In [ ]:
leafmap.maxar_collections()

Retrieve all collections for a specific event:

In [ ]:
collections = leafmap.maxar_child_collections('Morocco-Earthquake-Sept-2023')
print(f"The number of collections: {len(collections)}")

In [ ]:
collections.sort()
collections

Retrieve all items (tiles) for a specific collection and generate the footprints:

In [ ]:
gdf = leafmap.maxar_items(
    collection_id='Morocco-Earthquake-Sept-2023', 
    child_id='10300100ECC53700', 
    return_gdf=True, 
    assets=['visual'])
gdf.head()

Add the footprints to the map:

In [ ]:
m = leafmap.Map()
m.add_gdf(gdf, layer_name="Footprints", zoom_to_layer=True, info_mode="on_click")
m

Retrieve the COG URLs for all tiles in a collection:

In [ ]:
images = gdf['visual'].tolist()
images[:5]

Download the COGs to a local directory:

In [ ]:
leafmap.maxar_download(images[:2])

Create a mosaic json file for the collection. You need to install `cogeo-mosaic` first using `pip install cogeo-mosaic`. Creating a mosaic json file might take a few minutes. Please be patient.

In [ ]:
# leafmap.create_mosaicjson(images, output='10300100ECC53700.json')

Make the mosaic json file available on the web, then you can add the mosaic to the map:

In [ ]:
m = leafmap.Map(height='600px')
m.add_basemap('SATELLITE')
url = 'https://open.gishub.org/maxar-open-data/datasets/Morocco-Earthquake-Sept-2023/10300100ECC53700.json'
m.add_stac_layer(url, name="Mosaic")
m.add_gdf(gdf, layer_name="Footprints", info_mode="on_click")
m

Retrieve the footprint of all tiles for a specific event. This might take 15+ minutes. Please be patient.

In [ ]:
# gdf = leafmap.maxar_all_items(
#     collection_id='Morocco-Earthquake-Sept-2023',
#     return_gdf=True, 
#     verbose=True
#     )
# gdf.to_file('maxar_footprints.geojson', driver='GeoJSON')
# gdf

Add the footprints to the map:

In [ ]:
m = leafmap.Map(center=[31.35874, -8.78226], zoom=8)
url = 'https://open.gishub.org/maxar-open-data/datasets/Morocco-Earthquake-Sept-2023.geojson'
m.add_geojson(url, layer_name="Footprints", info_mode="on_click")
m

You can find the list of all available images at [Morocco-Earthquake-Sept-2023.tsv](https://github.com/opengeos/maxar-open-data/blob/master/datasets/Morocco-Earthquake-Sept-2023.tsv).

In [ ]:
gdf = leafmap.geojson_to_gdf(url)
gdf.head()

In [ ]:
before = gdf[gdf['datetime'] < '2023-09-10']
len(before)

In [ ]:
before.head()

In [ ]:
after = gdf[gdf['datetime'] >= '2023-09-10']
len(after)

In [ ]:
after.head()

In [ ]:
m = leafmap.Map()
m.add_gdf(before, layer_name="Before", info_mode="on_click")
style_dict = {'color': 'red', 'fillColor': 'red', 'fillOpacity': 0.1}
style_function = lambda x: style_dict
m.add_gdf(after, layer_name="After", info_mode="on_click", style_function=style_function)
m